In [7]:
import tensorflow as tf


# import neptune
# from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator_categorical import WindowGenerator

from run_automation_utils import Params, init_model

from typing import Dict

In [8]:
def prep_window_generator(station_id):
    df = pd.read_feather(f"../../data/pollution/processed/categorised/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    df["kategorie"] = df["kategorie"].astype(np.int32)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["kategorie"])

In [9]:
window_generator_538 = prep_window_generator(538)

In [10]:
CONV_WIDTH = 3
OUT_STEPS = 24  # 14 days prediction
pred_categories = 6  # TODO: ??? num_features is 11 but must be one for our prediction

cnn_lstm_model = tf.keras.models.Sequential()
# Shape [batch, time, features] => [batch, CONV_WIDTH, features]
cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]))
# Shape => [batch, 1, conv_units]
cnn_lstm_model.add(tf.keras.layers.Conv1D(256, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))

cnn_lstm_model.add(tf.keras.layers.Flatten())
# LSTM
cnn_lstm_model.add(tf.keras.layers.LSTM(32, return_sequences=True))
cnn_lstm_model.add(tf.keras.layers.LSTM(32, return_sequences=False))

cnn_lstm_model.add(tf.keras.layers.Dense(512))
cnn_lstm_model.add(tf.keras.layers.Dropout(0.4))
# Shape => [batch, out_steps, features] -> for each prediction step one neuron
# Anpassung für die Klassifizierung
cnn_lstm_model.add(tf.keras.layers.Dense(OUT_STEPS * pred_categories, activation="softmax"))

# Shape => [batch, out_steps, pred_categories]
cnn_lstm_model.add(tf.keras.layers.Reshape([OUT_STEPS, pred_categories]))

In [11]:
cnn_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"]
)

In [12]:
cnn_lstm_model.fit(window_generator_538.train, epochs=10, validation_data=window_generator_538.val)

(None, 24, 6, 1)


ValueError: in user code:

    File "c:\Users\nikla\Programming\FH Bielefeld\DataMining\feinstaubprojekt-polen\src\model\window_generator_categorical.py", line 62, in split_window  *
        labels.set_shape([None, self.label_width, 6])

    ValueError: Shapes must be equal rank, but are 4 and 3
